In [15]:
import pandas as pd
import re

# Load data
try:
    df = pd.read_csv('master.csv')
    df['MO Type'] = df['MO Type'].astype(str)
    df['Checking Attribute'] = df['Checking Attribute'].astype(str)
    print("master.csv loaded successfully.")
except FileNotFoundError:
    df = pd.DataFrame(columns=['MO Type', 'Checking Attribute'])
    print("master.csv not found. Starting with an empty DataFrame.")
except Exception as e:
    df = pd.DataFrame(columns=['MO Type', 'Checking Attribute'])
    print(f"Error loading master.csv: {e}. Starting with an empty DataFrame.")


master.csv loaded successfully.


In [16]:
awaiting_filter_value = False
filter_by = None
clarification_needed = False
filtered_data = None

In [21]:
print("Welcome to the Rego Policy Generator Chat!")
print("Type 'exit' to quit the chat.\n")

while True:
    user_input = input("You: ").strip()
    
    if user_input.lower() == 'exit':
        print("Goodbye!")
        break
        
    # Process empty input
    if not user_input:
        print("Bot: Please type a message.")
        continue
    
    # State 1: Awaiting filter value after clarification
    if awaiting_filter_value:
        filter_value = user_input
        awaiting_filter_value = False
        
        if filter_by == 'mo type':
            filtered_df = df[df['MO Type'].str.contains(filter_value, case=False, na=False)]
        elif filter_by == 'checking attribute':
            filtered_df = df[df['Checking Attribute'].str.contains(filter_value, case=False, na=False)]
        elif filter_by == 'both':
            filtered_df = df[
                df['MO Type'].str.contains(filter_value, case=False, na=False) |
                df['Checking Attribute'].str.contains(filter_value, case=False, na=False)
            ]
        
        if not filtered_df.empty:
            filtered_data = filtered_df.to_dict(orient='records')
            print("Bot: Filtered data found. Now I can generate the Rego policy. Would you like to generate it? (yes/no)")
        else:
            print("Bot: No data found matching your criteria. Please try again.")
        continue
    
    # Check for rego policy generation intent
    rego_keywords = ['rego', 'policy', 'generate', 'create', 'make']
    if any(keyword in user_input.lower() for keyword in rego_keywords):
    
        mo_type_value = None
        checking_attribute_value = None
        
        for val in df['MO Type'].dropna().unique():
            if re.search(r'\b' + re.escape(val.lower()) + r'\b', user_input.lower()):
                mo_type_value = val
                break
                
        for val in df['Checking Attribute'].dropna().unique():
            if re.search(r'\b' + re.escape(val.lower()) + r'\b', user_input.lower()):
                checking_attribute_value = val
                break
        
        if mo_type_value and checking_attribute_value:
            filter_by = 'both'
            awaiting_filter_value = True
            print(f"Bot: Found MO Type: {mo_type_value} and Checking Attribute: {checking_attribute_value}. Proceeding to filter.")
        elif mo_type_value:
            filter_by = 'mo type'
            awaiting_filter_value = True
            print(f"Bot: Found MO Type: {mo_type_value}. Proceeding to filter.")
        elif checking_attribute_value:
            filter_by = 'checking attribute'
            awaiting_filter_value = True
            print(f"Bot: Found Checking Attribute: {checking_attribute_value}. Proceeding to filter.")
        else:
            clarification_needed = True
            print("Bot: To generate the Rego policy, I need to filter the data. What kind of parameter are you looking for? (MO Type, Checking Attribute, or Both)")
        continue
    
    # State 3: User clarifies parameter type
    elif user_input.lower() in ['mo type', 'checking attribute', 'both'] and clarification_needed:
        clarification_needed = False
        filter_by = user_input.lower()
        awaiting_filter_value = True

        suggestions = []
        if user_input.lower() == 'mo type':
            suggestions = df['MO Type'].dropna().unique().tolist()
        elif user_input.lower() == 'checking attribute':
            suggestions = df['Checking Attribute'].dropna().unique().tolist()
        elif user_input.lower() == 'both':
            suggestions = list(set(df['MO Type'].dropna().unique().tolist() + 
                              df['Checking Attribute'].dropna().unique().tolist()))

        response_message = f'You selected to filter by {user_input}. Please provide the value for {user_input}.'
        if suggestions:
            response_message += f' Some suggestions: {", ".join(suggestions[:5])}...'
        print(f"Bot: {response_message}")
        continue
    
    # State 4: User confirms Rego generation
    elif user_input.lower() == 'yes' and filtered_data:
        rego_policy = "# Rego Policy Generated\n\n"
        for row in filtered_data:
            mo_type = row.get('MO Type', 'N/A')
            checking_attribute = row.get('Checking Attribute', 'N/A')
            rego_policy += f"package ericsson.consistency.{mo_type.lower().replace(' ', '_')}\n\n"
            rego_policy += f"default allow = false\n\n"
            rego_policy += f"allow {{\n"
            rego_policy += f"    input.mo_type == \"{mo_type}\"\n"
            rego_policy += f"    input.checking_attribute == \"{checking_attribute}\"\n"
            rego_policy += f"}}\n\n"
        filtered_data = None
        print(f"Bot: Here is your Rego policy:\n\n{rego_policy}")
        continue
    
    elif user_input.lower() == 'no':
        print("Bot: Okay, let me know if you want to generate a policy later.")
        continue
    
    # Default response
    else:
        print("Bot: I'm sorry,I am not able provide answere with the input details.")


Welcome to the Rego Policy Generator Chat!
Type 'exit' to quit the chat.

Bot: To generate the Rego policy, I need to filter the data. What kind of parameter are you looking for? (MO Type, Checking Attribute, or Both)
Bot: You selected to filter by checking attribute. Please provide the value for checking attribute. Some suggestions: qciTab8boostFactorUl, interferenceAveragingProcessBoundary5, mtSIMonWBTS, dllaTargetBlerVoNR, DCHUtilUpperDowngradeThr256...
Bot: Filtered data found. Now I can generate the Rego policy. Would you like to generate it? (yes/no)
Bot: Here is your Rego policy:

# Rego Policy Generated

package ericsson.consistency.lnbts

default allow = false

allow {
    input.mo_type == "LNBTS"
    input.checking_attribute == "qciTab8boostFactorUl"
}


Bot: I'm sorry, I can only generate Rego policies for parameter consistency checking for Ericsson data models at the moment.


KeyboardInterrupt: Interrupted by user